In [1]:
#In this program we are gonna classify ten images of CIFAR dataset most famous.
# Initially we are gonna import data and and then do the image classification using CNN
# Due to system limitations much optimization cant be done and hence we would suffice with four layers
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns #for making heatmaps



In [3]:
# For transforming tensors
transform = transforms.ToTensor()

train_data = datasets.CIFAR10(root="../data..",download=True,train=True,transform=transform)
test_data = datasets.CIFAR10(root="../data..",download=True,train=False,transform=transform)

In [4]:
train_data

In [5]:
test_data

In [6]:
train = DataLoader(train_data,shuffle=True,batch_size=10)
test = DataLoader(test_data,shuffle=False,batch_size=10)

In [7]:
class_names = ['plane', '  car', ' bird', '  cat', ' deer', '  dog', ' frog', 'horse', ' ship', 'truck']
for image,labels in train:
    break
#print the labels
print("labels:",labels.numpy())
print("Images:",*np.array([class_names[i] for i in labels]))


In [8]:
# Print the images
im = make_grid(image, nrow=5)  # the default nrow is 8
plt.figure(figsize=(10,4))
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)));

In [9]:
# Building the convolutional neural network model
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,6,5,1)
        self.conv2 = nn.Conv2d(6,16,5,1)
        self.fc1= nn.Linear(16*5*5,200)
        self.fc2 = nn.Linear(200,80)
        self.fc3 = nn.Linear(80,10)
        
    

    def forward(self,x):
        x = f.relu(self.conv1(x))
        x= f.max_pool2d(x,2,2)
        x = f.relu(self.conv2(x))
        x = f.max_pool2d(x,2,2)
        x = torch.flatten(x,1)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
    
        return f.log_softmax(x, dim=1)



In [10]:
model = ConvolutionalNeuralNetwork()
model

In [11]:
# Identifying criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [12]:
#Training the model
import time
start_time = time.time()
epochs = 10
train_losses = []
test_losses = []
train_correct = []
test_correct = []
for i in range(epochs):
    trn_corr= 0
    tst_corr = 0
    for b,(X_train,y_train) in enumerate(train):
        b+=1
        #Applying for the model
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)

        #Tally the number of correct predictions 
        predicted = torch.max(y_pred.data,1)[1]
        batch_corr = (predicted==y_train).sum()
        trn_corr += batch_corr
        #Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Print interim results 
        if b%1000 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/50000]  loss: {loss.item():10.8f}  \ accuracy: {trn_corr.item()*100/(10*b):7.3f}%')
    loss=loss.detach().numpy()
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    #Run the testing gradient
    
    with torch.no_grad():
        for b,(X_test,y_test) in enumerate (test):
            y_val = model(X_test)
            loss = criterion(y_val,y_test)
            #Tally the number of corrected predictions
            predicted = torch.max(y_val.data,1)[1]
            tst_corr += (predicted==y_test).sum()

        loss=loss.detach().numpy()
        test_losses.append(loss)
        test_correct.append(tst_corr)

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

        



In [13]:
# Plotting the train and test losses
plt.plot(train_losses, label='training loss')
plt.plot(test_losses, label='validation loss')
plt.title('Loss at the end of each epoch')
plt.legend();

In [14]:
#Testing whether the algorithm classified correctly or not
img,data = test_data[450]
plt.imshow(img.permute(1,2,0))

In [15]:
print('Label:', test_data.classes[data])
print(img.shape)

In [16]:
#Testing with individual images
class_names = ['plane', '  car', ' bird', '  cat', ' deer', '  dog', ' frog', 'horse', ' ship', 'truck']
model.eval()
x,y = test_data[225][0], test_data[225][1]
with torch.no_grad():
    pred_1 = model(x.reshape(1,3,32,32))
    predicted, actual = class_names[pred_1[0].argmax(0)], class_names[y]
    print(f"Predicted: {predicted}, Actual: {actual}")
    


In [17]:
class_names = ['plane', '  car', ' bird', '  cat', ' deer', '  dog', ' frog', 'horse', ' ship', 'truck']
model.eval()
x1,y1 = test_data[155][0],test_data[155][1]
with torch.no_grad():
    pred_2 = model(x1.reshape(1,3,32,32))
    predicted,actual = class_names[pred_2[0].argmax(0)], class_names[y1]
    print(f"Predicted: {predicted}, Actual: {actual}")

In [18]:
class_names = ['plane', '  car', ' bird', '  cat', ' deer', '  dog', ' frog', 'horse', ' ship', 'truck']
model.eval()
x2,y2 = test_data[15][0], test_data[15][1]
with torch.no_grad():
    pred_2 = model(x2.reshape(1,3,32,32))
    predicted,actual = class_names[pred_2[0].argmax(0)], class_names[y2]
    print(f"Predicted: {predicted}, Actual: {actual}")